#  Example of the processing of a unique image 
This example shows the image processing of the confocal image of the top surface of a raw silicon wafer from diamond wire cutting. The aim is to extract and characterize in terms of size and location the topography "features"  induced by diamonds indentation or fragile fracture present on the wafer surface and superimposed with saw marks.
   

In [ ]:
# Standard library imports
import os
from pathlib import Path

# Path of 'site-packages' where useful packages are stored on MAC-OS
mac_packages = "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages" 

# Root directory for all files 
root = Path("C:/Users/franc/OneDrive/Bureau/confocal/fichiers")
if not os.path.isdir(root) : 
    root = Path('/Users/amal/Gwyddion Images /Carton-Louise') 
store_file = root / Path('synthesis.xlsx')

# Parameters of image reference 
cut_number = '119'
wafer_number = '5'
image_ref = 'd'

# Specification of the directory of the experimental files and useful file names
root_input = root / Path('cut ' + cut_number + '/wafer '+ wafer_number + '/save_plu')
root_output = root / Path('cut ' + cut_number + '/wafer '+ wafer_number + '/results')
if not os.path.isdir(root_output) :
    os.mkdir(root_output)

# image_name with two possible spelling
image_name = 'wafer' + wafer_number + '-' + image_ref        # input file
input_file_name = root_input / Path(image_name + '.plu') 
if not os.path.isfile(input_file_name) : # deal with additional blanck
    blk = ' '
    image_name = 'wafer' + blk + wafer_number + '-' + image_ref   # alternative input file
    input_file_name = root_input / Path(image_name + '.plu') 
    
file_xlsx_res = root_output / Path(image_name + '_res.xlsx') # file EXEL features morphological analysis


In [ ]:
'''
Generated image processing:
- saw marks removal and background correction using a top hat filter with a pattern length
larger than the largest feature to extract
- image binarization using a threshold on feature depth larger than the residual image background
- features morphological analysis (using the scipy label function). Generate a
        dataframe  index|'x'|'long_x'| 'y'|'long_y'| 'size'|'depth' where
            - index: feature index
            - x: gravity center x position of the feature
            - long_x: maximum feature width
            - y: gravity center y position of the feature
            - long_y: maximum feature height
            - size: pixels number of the feature 
            - depth : feature depth

Plot of the results (images, histogram, box plot)

Store the results in an excel file

'''
# 3rd party imports
import numpy as np
import matplotlib.colors as colx
import matplotlib.cm as cmx
import matplotlib.pyplot as plt

# Add package image_features_extract
try: # standard storage path of 'site-packages' on WIN
    import image_features_extract as ife
except: # Add storage path of 'site-packages' on MAC-OS
    import sys
    sys.path.append(mac_packages)
    import image_features_extract as ife

%matplotlib inline

store_flag = True # if store_flag = True we store the resuts in an excel file
unmeasured_point_color = 'blue' # color of dots representing the unmeasured points

# parameters initialization
Top_hat_length = 50 # should be larger than the largest feature to extract
threshold = -0.3    # should be larger than the residual image background


# read .plu image (N rows by M columns)
N, M, confocal_img, _ = ife.read_plu_topography(input_file_name)


# substitute unmeasured values by interpolated ones
confocal_img1 = ife.fill_gap(confocal_img)


im_corr = ife.top_hat_flattening(confocal_img1, Top_hat_length, top_hat_sens=-1 )# background removal

im_bin = np.where(im_corr < threshold, 1, 0) # binarization

df = ife.analyse_morphologique(im_bin,im_corr) # morphological analysis stored in df dataframe

fig = plt.figure(figsize=(15,15))

# stacked plot of the flattened image with dots at the position of the unmeasured points
plt.subplot(3,2,1)
plt.imshow(confocal_img1, cmap='gray')
plt.colorbar()
mask = np.where(confocal_img == 1000001)
plt.scatter(mask[1],mask[0],c=unmeasured_point_color,s =1,alpha=0.1 )
plt.title(f'Confocal image and unmeasured points ({unmeasured_point_color})')

# top-hat filtered image plot
plt.subplot(3,2,2)
plt.imshow(im_corr, cmap='gray')
plt.colorbar()
plt.title(f'Top hat length: {Top_hat_length}')

# binarized image plot
plt.subplot(3,2,3)
plt.imshow(im_bin, cmap='gray')
plt.colorbar() 
plt.title(f'Threshold: {threshold} µm')

# reconstructed image plot
ax1 = plt.subplot(3,2,4)
colorsMap = plt.get_cmap("plasma") #("RdYlGn")
g = list(zip(np.array(df['x']),
             np.array(df['y']),
             np.array(df['height']),
             np.array(df['size'])))
x,y,c,s=zip(*sorted(g, key=lambda tup: tup[2],reverse=False))

cm = plt.get_cmap(colorsMap)
cNorm = colx.Normalize(vmin=min(c), vmax=max(c))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)

conv_inch_pixel = 5 # sloppy determination, must be improved
plt.scatter(np.array(x) ,N-np.array(y) ,s=np.array(s)/conv_inch_pixel, c=scalarMap.to_rgba(c))
plt.xlim(0,M)
plt.ylim(0,N)
trash = plt.yticks(ticks=[N,N-100, N-200, N-300, N-400, N-500],labels=[0,100,200,300,400,500],
                               rotation=0)
scalarMap.set_array(c)
plt.colorbar(scalarMap)
plt.title("Size and height")

# features size histogram plot
plt.subplot(3,2,5)
h = plt.hist(df['size'], bins=100)
plt.title(f'Size histogram')
plt.ylabel('size (pix\u00B2)')

# features size box plot
plt.subplot(3,2,6)
h = plt.boxplot(df['size'])
plt.title(f'Size bbox')
_ = plt.ylabel('size (pix\u00B2)')

# conditional storage of morphological analysis result
if store_flag:  
    df.to_excel(file_xlsx_res)

In [ ]:
'''
# Plot of selected horizontal and vertical profiles from the top-hat filtered image

'''

# 3rd party imports
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

index =[ 50] # list of the row or column index to be plotted

fig = plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
color =['k','r','b','y']
for c,index_  in zip(color,index):
    plt.plot(confocal_img1.T[index_],'-'+c, label=str(index_))
    plt.plot(im_corr.T[index_],'--'+c, label='cor '+str(index_))
plt.ylim(np.min(confocal_img1),np.max(confocal_img1))
plt.legend()
plt.ylabel('Height (µm)')
plt.title("Vertical profiles")

plt.subplot(1,2,2)
for c,index_  in zip(color,index):
    plt.plot(confocal_img1[index_],'-'+c,label=str(index_))
    plt.plot(im_corr[index_],'--'+c, label='cor '+str(index_))
plt.ylim(np.min(confocal_img1),np.max(confocal_img1))
plt.legend()
plt.ylabel('Height (µm)')
_ = plt.title("Horizontal profiles")